In [95]:
import sys
sys.path.insert(0, '../gofher')

import os
import matplotlib.image as mpimg
import numpy as np

from gofher import run_gofher, run_gofher_with_parameters
from visualize import visualize
from sparcfire import read_sparcfire_galaxy_csv, get_ref_band_and_gofher_params
from file_helper import write_csv,check_if_folder_exists_and_create
from spin_parity import read_spin_parity_galaxies_label_from_csv, standardize_galaxy_name

In [96]:
survery_to_use = "sdss"

BANDS_IN_ORDER = ['g','r','i','z'] #Important: Must stay in order of BLUEST to REDDEST Waveband (Editting this will cause gofher to no longer correctly evaluate redder side of galaxy)
REF_BANDS_IN_ORDER = ['r','i','z','g'] #The prefernce each waveband being choosen as refernce band from highest priority to lowest priority

In [97]:
figure_to_run_on = "figure11"

In [98]:
run_type = "fixed" #["inital","fixed-center","fixed","source_extraction"]
bulge_disk_f = 0.125

substring = run_type
if run_type != "": substring += "_{}".format(str(bulge_disk_f).replace(".","_"))

In [99]:
generate_verbose_csv = True
generate_ebm_csv = False

generate_visualization = True
save_visualization = True

In [100]:
#Important: Make sure you update these values:
path_to_catalog_data = "..\\..\\spin-parity-catalog-data"
path_to_output = "..\\..\\gofher-data\\{}\\{}".format(survery_to_use,substring)

In [101]:
def get_fits_path(name,band):
    """the file path of where existing fits files can be found"""
    return os.path.join(path_to_catalog_data,survery_to_use,figure_to_run_on,name,"{}_{}.fits".format(name,band))

def get_color_image_path(name):
    return os.path.join(path_to_catalog_data,survery_to_use,figure_to_run_on,name,"{}_color.png".format(name))

def get_path_to_catalog_csv():
    return os.path.join(path_to_catalog_data,"catalog","{}.csv".format(figure_to_run_on))

In [102]:
def get_paper_dark_side_labels():
    return read_spin_parity_galaxies_label_from_csv(get_path_to_catalog_csv())

def get_galaxies():
    return os.listdir(os.path.join(path_to_catalog_data,survery_to_use,figure_to_run_on))

def get_sparcfire_path(): #temp
    table_dict = {"figure8":"2","figure9":"3","figure10":"4","figure11":"5"}
    pa_key = table_dict.get(figure_to_run_on,"")
    if pa_key == "": raise ValueError("Invlid figure to run on")

    return "C:\\Users\\school\\Desktop\\cross_id\\sdss_mosaic_construction\\SpArcFiRe_output\\table{}\\G.out\\galaxy.csv".format(pa_key)

In [103]:
def run_gofher_on_catalog():
    paper_labels = get_paper_dark_side_labels()

    if run_type != "":
        sparcfire_gals = read_sparcfire_galaxy_csv(get_sparcfire_path())

    verbose_header = []
    verbose_rows = []

    ebm_header = []
    ebm_rows = []

    i = 1

    for name in get_galaxies():

        if standardize_galaxy_name(name) not in paper_labels:
            print("skippimg",name)
            continue

        print(name, i,"of",len(get_galaxies()))

        if run_type != "" and name not in sparcfire_gals:
            print("skipping")
            continue

        try:
            paper_label = paper_labels[standardize_galaxy_name(name)]

            if run_type == "source_extraction":
                gal = run_gofher(name,get_fits_path,BANDS_IN_ORDER,REF_BANDS_IN_ORDER, paper_label)
            else:
                ref_band, inital_gofher_params = get_ref_band_and_gofher_params(sparcfire_gals[name],REF_BANDS_IN_ORDER,bulge_disk_f)
                gal = run_gofher_with_parameters(name,get_fits_path,BANDS_IN_ORDER,ref_band,inital_gofher_params,paper_label=paper_label,mode=run_type)


            if generate_verbose_csv:
                (header,row) = gal.get_verbose_csv_header_and_row(BANDS_IN_ORDER,paper_label)
                if len(verbose_header) == 0: verbose_header = header
                verbose_rows.append(row)

            if generate_ebm_csv:
                (header,row) = gal.get_ebm_csv_header_and_row(BANDS_IN_ORDER, paper_label)
                if len(ebm_header) == 0: ebm_header = header
                ebm_rows.append(row)

            if generate_visualization:
                save_path = ''
                
                if save_visualization:
                    sub_folder = os.path.join(path_to_output,figure_to_run_on)
                    check_if_folder_exists_and_create(sub_folder)
                    save_path = os.path.join(sub_folder,"{}.png".format(name))

                color_image = mpimg.imread(get_color_image_path(name))
                color_image = np.flipud(color_image) #flip the image left to right to fix orientation
                visualize(gal,color_image,BANDS_IN_ORDER,paper_label,save_path=save_path)
        except Exception as e:
            print(e)
        i += 1

    if generate_verbose_csv:
        verbose_csv_path = os.path.join(path_to_output,"{}_verbose.csv".format(figure_to_run_on))
        write_csv(verbose_csv_path,verbose_header,verbose_rows)

    if generate_ebm_csv:
        ebm_csv_path = os.path.join(path_to_output,"{}_ebm.csv".format(figure_to_run_on))
        write_csv(ebm_csv_path,ebm_header,ebm_rows)

In [104]:
if not os.path.exists(path_to_catalog_data):
    raise ValueError("The path to the catalog is not found {} - make sure you update path_to_catalog_data".format(path_to_catalog_data))

if not os.path.exists(path_to_output):
    make_ouput_folder_if_not_exists = True

    if not make_ouput_folder_if_not_exists:
        raise ValueError("The path output is not found {} - make sure you update path_to_output".format(path_to_output))
    os.makedirs(path_to_output)


run_gofher_on_catalog()

IC2247 1 of 25
IC540 2 of 25
IC944 3 of 25
MCG-02-02-040 4 of 25
skipping
MCG-02-03-015 4 of 25
skipping
NGC1542 4 of 25
NGC3067 5 of 25
NGC3079 6 of 25
NGC3169 7 of 25
NGC3495 8 of 25
NGC3626 9 of 25
NGC4517 10 of 25
zero-size array to reduction operation minimum which has no identity
NGC4605 11 of 25
NGC4772 12 of 25
NGC6314 13 of 25
NGC681 14 of 25
UGC10205 15 of 25
UGC10297 16 of 25
UGC3107 17 of 25
UGC5111 18 of 25
UGC5498 19 of 25
UGC6036 20 of 25
UGC8267 21 of 25
UGC8778 22 of 25
UGC9665 23 of 25
